In [ ]:
!pip install keras
!pip install tensorflow --upgrade
!pip uninstall tensorflow
!pip install tensorflow==2.12.0
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from tensorflow.keras import Sequential

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:

pd.options.display.max_rows = 999

pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [ ]:
spotify_df = pd.read_csv('E:\Capstone DS\Mood\data_moods.csv')

In [ ]:
spotify_df.dropna(subset=['consolidates_genre_lists'], inplace=True)

In [ ]:
spotify_df.isna().sum()

In [ ]:
spotify_df.head()

In [ ]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [ ]:
col_features = mood_prep.columns[:]
col_features

In [ ]:
Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [ ]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [ ]:
mood_trans_np[10011]

In [ ]:
array([ 3.37093e+05,  4.26000e-01,  1.01000e-01,  6.46000e-01,
        0.00000e+00,  3.58000e-01,  8.21000e-01, -1.10330e+01,
        1.32000e-01,  2.05703e+02])

In [ ]:
df = pd.read_csv('df = pd.read_csv('data_moods.csv')

In [ ]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

In [ ]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [ ]:
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

In [ ]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict()

In [ ]:
y_preds

In [ ]:
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
pip.fit(X2,encoded_y)

In [ ]:
def predict_mood(preds):

In [ ]:
preds_features = np.array(preds[:]).reshape(-1,1).T

In [ ]:
results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])

In [ ]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

In [ ]:
spotify_df.shape

In [ ]:
print(len(res))

In [ ]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [ ]:
spotify_df.to_csv('MusicMoodFinal.csv')

In [ ]:
res.count("Sad")

In [ ]:
res.count("Happy")

In [ ]:
res.count("Energetic")

In [ ]:
res.count("Calm")

In [ ]:
spotify_df.shape